In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
StandardScaler()
from sklearn.preprocessing import OrdinalEncoder
OrdinalEncoder()

OrdinalEncoder()

In [2]:
# load training data and test data
data = pd.read_csv('adult/adult.data', header = None)
# data = pd.read_csv('adult/adult.test', header = None, skiprows=1)

# set column name
data.columns = ['age','workclass','fnlwgt','education','education-num','marital-status','occupation','relationship','race','sex','capital-gain','capital-loss','hours-per-week','native-country','income']
data.head(10)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
5,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K
6,49,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,<=50K
7,52,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,>50K
8,31,Private,45781,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50,United-States,>50K
9,42,Private,159449,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,United-States,>50K


In [3]:
# create df as copy of data
df = data.copy(deep = True)

# remove outlier " ?" and null values
for col in data.columns:
    df[[col]] = data[[col]].replace(' ?',np.NaN)
df = df.dropna()

In [4]:
# get all column names for numerical attributes
colname= []
for col in df.columns:
    if df[col].dtype == "int64":
        colname.append(col)
        
# create df only including categorical attributes
df_copy = df.copy(deep = True)
de_ctgr = df.copy()
# drop label "income"
de_ctgr.drop('income', axis='columns', inplace=True) 
# drop numerical attributes
de_ctgr.drop(colname, axis='columns', inplace=True) 

# create df only including label
df_label = df.copy()
df_label.drop(df_copy.iloc[:,0:14],axis='columns', inplace=True)

# normalize numerical attributes
df_norm = pd.DataFrame(StandardScaler().fit_transform(df_copy[colname]), columns=colname)

In [5]:
# encode categorical attributes into numerical attributes
for col in de_ctgr.columns[:]:
    if de_ctgr[col].dtype == "object":
        de_ctgr[col] = OrdinalEncoder().fit_transform(de_ctgr[[col]])

# set " >50K" to 0 and " <=50K" to 1
df_label["income"] = np.where(df_label["income"].str.contains(">50K"), 0, 1)

In [6]:
# merge three df as new dataset
# Insert id column to different datasets
df_norm.insert(0, 'id', range(0, 0 + len(df_norm)))
de_ctgr.insert(0, 'id', range(0, 0 + len(de_ctgr)))
df_label.insert(0, 'id', range(0, 0 + len(df_label)))

# merge three df
de_catogory = pd.merge(df_norm,de_ctgr, on=["id"])
df_result = pd.merge(de_catogory,df_label, on=["id"])

# drop id column
df_result.drop('id',axis='columns', inplace=True)
df_result.head(10)

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,workclass,education,marital-status,occupation,relationship,race,sex,native-country,income
0,0.042796,-1.062722,1.128918,0.146092,-0.218586,-0.077734,5.0,9.0,4.0,0.0,1.0,4.0,1.0,38.0,1
1,0.880288,-1.007871,1.128918,-0.147445,-0.218586,-2.331531,4.0,9.0,2.0,3.0,0.0,4.0,1.0,38.0,1
2,-0.033340,0.244693,-0.439738,-0.147445,-0.218586,-0.077734,2.0,11.0,0.0,5.0,1.0,4.0,1.0,38.0,1
3,1.108695,0.425240,-1.224066,-0.147445,-0.218586,-0.077734,2.0,1.0,2.0,5.0,0.0,2.0,1.0,38.0,1
4,-0.794697,1.406658,1.128918,-0.147445,-0.218586,-0.077734,2.0,9.0,2.0,9.0,5.0,2.0,0.0,4.0,1
5,-0.109476,0.897180,1.521083,-0.147445,-0.218586,-0.077734,2.0,12.0,2.0,3.0,5.0,4.0,0.0,38.0,1
6,0.804152,-0.280232,-2.008395,-0.147445,-0.218586,-2.081109,2.0,6.0,3.0,7.0,1.0,2.0,0.0,22.0,1
7,1.032559,0.187865,-0.439738,-0.147445,-0.218586,0.339636,4.0,11.0,2.0,3.0,0.0,4.0,1.0,38.0,0
8,-0.566290,-1.363097,1.521083,1.754199,-0.218586,0.757005,2.0,12.0,4.0,9.0,1.0,4.0,0.0,38.0,0
9,0.271203,-0.287217,1.128918,0.551697,-0.218586,-0.077734,2.0,9.0,2.0,3.0,0.0,4.0,1.0,38.0,0


In [7]:
# upgrade data file
df_result.to_csv("adult/trainData.csv", index=None)
# df_result.to_csv("adult/testData.csv", index=None)